In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import matplotlib.image as mpimg
import glob, os
from pathlib import Path
from IPython.core.display import display,HTML
pd.set_option('display.max_colwidth', None)

In [2]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [3]:
labels = pd.read_csv('datasets/mikey-devon-labels_validation.csv', delimiter=";")

In [5]:
a = labels['tag_list'].str.contains(",")
labels['multiple_tags'] = a
labels.loc[labels['multiple_tags'] == True]

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,multiple_tags
8,Devon Snyder,10,481,University District,Signal,47.661236,-122.320930,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:03:18.775-07,t,NaN,f,"button waist height,has button",NaN,True
9,Devon Snyder,11,481,University District,Signal,47.661209,-122.321114,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:03:33.652-07,t,NaN,f,"has button,button waist height",NaN,True
16,Devon Snyder,18,482,University District,Signal,47.661438,-122.320831,j3GefSTHZmrn2LHTGg8SFA,2022-05-10 14:08:04.76-07,t,NaN,f,"button waist height,has button",NaN,True
17,Devon Snyder,19,482,University District,CurbRamp,47.661415,-122.320831,j3GefSTHZmrn2LHTGg8SFA,2022-05-10 14:08:11.854-07,t,3.0,f,"not enough landing space,narrow,surface problem",NaN,True
20,Devon Snyder,22,482,University District,CurbRamp,47.661488,-122.320854,qCkbp2lszdGrR5s3hdSiTQ,2022-05-10 14:09:52.251-07,t,2.0,f,"not enough landing space,surface problem",NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2326,mikey,2447,811,Windermere,SurfaceProblem,47.672169,-122.263733,rztI_Sg0PBzK1toWlOoneg,2022-07-06 13:38:15.033-07,NaN,2.0,f,"uneven/slanted,grass",NaN,True
2335,mikey,2456,811,Windermere,CurbRamp,47.672146,-122.264488,942Ur_vL2EhxfX2_o0xWqw,2022-07-06 13:40:38.753-07,NaN,5.0,f,"surface problem,missing tactile warning",driveway can be used,True
2339,mikey,2460,810,Windermere,SurfaceProblem,47.672195,-122.264603,9ZSR7dhIscScTkfcbX_zyQ,2022-07-06 13:43:58.314-07,NaN,1.0,f,"uneven/slanted,grass",NaN,True
2342,mikey,2464,810,Windermere,SurfaceProblem,47.672165,-122.265541,VtJzb6MCJ1nL96JGZG24sw,2022-07-06 13:46:33.6-07,NaN,2.0,f,"uneven/slanted,grass",NaN,True


In [9]:
#create labels geo dataframe
points = labels.apply(lambda row: Point(row.lng, row.lat), axis=1)
labels_geo = gpd.GeoDataFrame(labels, geometry=points)
labels_geo.crs = {'init': 'epsg:4326'}
labels_geo

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,1,480,University District,Occlusion,47.661045,-122.321037,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:02.905-07,NaN,NaN,f,NaN,NaN,POINT (-122.32104 47.66105)
1,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,POINT (-122.32071 47.66104)
2,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,POINT (-122.32070 47.66103)
3,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,POINT (-122.32071 47.66106)
4,Devon Snyder,6,480,University District,Occlusion,47.660900,-122.321007,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:52.042-07,NaN,NaN,f,NaN,NaN,POINT (-122.32101 47.66090)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,mikey,2493,810,Windermere,SurfaceProblem,47.672043,-122.267929,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:40.653-07,NaN,1.0,f,uneven/slanted,NaN,POINT (-122.26793 47.67204)
2372,mikey,2494,716,Windermere,Obstacle,47.671928,-122.268097,sOyfUL3XT8ETE-9XfeaXvQ,2022-07-06 14:14:34.691-07,NaN,1.0,f,vegetation,NaN,POINT (-122.26810 47.67193)
2373,mikey,2495,715,Windermere,NoSidewalk,47.672016,-122.268433,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:48.626-07,NaN,3.0,f,street has a sidewalk,NaN,POINT (-122.26843 47.67202)
2374,mikey,2496,716,Windermere,SurfaceProblem,47.671883,-122.268181,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:55.957-07,NaN,1.0,f,cracks,NaN,POINT (-122.26818 47.67188)


In [10]:
# Seperate Seattle and Chicago data
chicago = labels_geo.loc[labels_geo['lng'] > -105]
seattle = labels_geo.loc[labels_geo['lng'] < -105]

In [11]:
map_clusters(chicago,0)

In [18]:
def agree_rate(label_data,i):
    df = cluster_label_type_at_index(label_data,i)[2]
    df["clustered"] = df.duplicated(subset="cluster_id", keep=False)
    df1 = df[df['clustered'] == True]
    
    if len(df) == 0:
        return 0
    else: 
        return len(df1)/len(df)

In [18]:
agree_rate(seattle,0)

0.8494623655913979

In [19]:
agree_rate(seattle,2)

0.5764075067024129

In [20]:
agree_rate(chicago,0)

0.9302325581395349

In [21]:
agree_rate(chicago,2)

0.5229357798165137

In [27]:
j=[]
k=[]
l=[]
for i in [0,1,2,7]:
     j.append(agree_rate(seattle,i))
for i in [0,1,2,7]:
    k.append(agree_rate(chicago,i))
for i in [0,1,2,7]:
    l.append( label_types[i])
agree_rate_df = pd.DataFrame(list(zip(l,j, k)),
              columns=['label_type','seattle', 'chicago'])

In [29]:
agree_rate_df

,label_type,seattle,chicago
0,CurbRamp,0.849462,0.930233
1,NoSidewalk,0.670807,0.666667
2,Problem,0.576408,0.522936
3,NoCurbRamp,0.852792,0.000000


In [31]:
agree_rate(chicago,4)

0.5

In [19]:
def agreement_df(label_index_list):
    j=[]
    k=[]
    l=[]
    for i in label_index_list:
        j.append(agree_rate(seattle,i))
    for i in label_index_list:
        k.append(agree_rate(chicago,i))
    for i in label_index_list:
        l.append( label_types[i])
    agree_rate_df = pd.DataFrame(list(zip(l,j, k)),columns=['label_type','seattle', 'chicago'])
    df = agree_rate_df.style.format({
    'seattle': '{:,.2%}'.format,
    'chicago': '{:,.2%}'.format,})
    
    return df 

In [20]:
agreement_df([0,1,2,4,5,7])

,label_type,seattle,chicago
0,CurbRamp,76.14%,68.49%
1,NoSidewalk,65.88%,54.55%
2,Problem,50.95%,36.89%
3,SurfaceProblem,49.34%,35.04%
4,Obstacle,33.33%,40.00%
5,NoCurbRamp,84.26%,0.00%
